# All codes below had been done in 2 hacking days

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
root_dir = '/content/gdrive/My Drive/sprayer_robot/'

In [ ]:
%cd {root_dir}

/content/gdrive/My Drive/sprayer_robot


In [ ]:
%cd models/research/
!protoc object_detection/protos/*.proto --python_out=.
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .

/content/gdrive/My Drive/sprayer_robot/models/research
Processing /content/gdrive/My Drive/sprayer_robot/models/research
     |████████████████████████████████| 8.7MB 7.2MB/s 
     |████████████████████████████████| 358kB 40.9MB/s 
     |████████████████████████████████| 849kB 43.0MB/s 
     |████████████████████████████████| 61kB 7.7MB/s 
     |████████████████████████████████| 63.8MB 47kB/s 
     |████████████████████████████████| 2.0MB 36.2MB/s 
     |████████████████████████████████| 153kB 45.7MB/s 
     |████████████████████████████████| 829kB 32.5MB/s 
     |████████████████████████████████| 61kB 7.1MB/s 
     |████████████████████████████████| 51kB 7.1MB/s 
     |████████████████████████████████| 174kB 41.7MB/s 
     |████████████████████████████████| 1.1MB 39.9MB/s 
     |████████████████████████████████| 36.7MB 82kB/s 
     |████████████████████████████████| 102kB 11.6MB/s 
     |████████████████████████████████| 112kB 44.0MB/s 
  Created wheel for object-detection: filename=o

In [ ]:
import cv2
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import tensorflow as tf
import glob
import os
import io
import scipy.misc
from six import BytesIO

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
%matplotlib inline

In [ ]:
def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: the file path to the image

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  img_data = tf.io.gfile.GFile(path, 'rb').read()
  image = Image.open(BytesIO(img_data))
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [ ]:
#recover our saved model
pipeline_config = '/content/gdrive/My Drive/sprayer_robot/deploy/efficientDet_d3/pipeline_file.config'
#generally you want to put the last ckpt from training in here
model_dir = '/content/gdrive/My Drive/sprayer_robot/deploy/training/ckpt-2'
configs = config_util.get_configs_from_pipeline_file(pipeline_config)
model_config = configs['model']
detection_model = model_builder.build(
      model_config=model_config, is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(
      model=detection_model)
ckpt.restore(os.path.join('/content/gdrive/My Drive/sprayer_robot/deploy/training/ckpt-2'))


def get_model_detection_function(model):
  """Get a tf.function for detection."""

  @tf.function
  def detect_fn(image):
    """Detect objects in image."""

    image, shapes = model.preprocess(image)
    prediction_dict = model.predict(image, shapes)
    detections = model.postprocess(prediction_dict, shapes)

    return detections, prediction_dict, tf.reshape(shapes, [-1])

  return detect_fn

detect_fn = get_model_detection_function(detection_model)

In [ ]:
%cd {root_dir}

/content/gdrive/My Drive/sprayer_robot


In [ ]:
#map labels for inference decoding
label_map_path = configs['eval_input_config'].label_map_path
label_map = label_map_util.load_labelmap(label_map_path)
categories = label_map_util.convert_label_map_to_categories(
    label_map,
    max_num_classes=label_map_util.get_max_label_map_index(label_map),
    use_display_name=True)
category_index = label_map_util.create_category_index(categories)
label_map_dict = label_map_util.get_label_map_dict(label_map, use_display_name=True)

In [ ]:
def get_bboxes(full_bboxes, full_scores, im_width, im_height, threshold = 0.5):
  bboxes = []
  for i in range(len(full_bboxes)):
    if full_scores[i] > threshold:
      y_min, x_min, y_max, x_max =  full_bboxes[i].tolist()
      (y_min, x_min, y_max, x_max) = (y_min * im_height, x_min * im_width, \
                                    y_max * im_height, x_max * im_width)
      bboxes.append((y_min, x_min, y_max, x_max))
  bboxes = np.array(bboxes)
  bboxes = bboxes.astype('int')
  return bboxes

In [ ]:
#run detector on test image
#it takes a little longer on the first run and then runs at normal speed. 
video_dir = os.path.join(root_dir, 'videos')
video_name = glob.glob(video_dir+'/*.mp4')[1]
output_dir = os.path.join(root_dir, 'output_videos')
output_name = os.path.join(output_dir, video_name.split('/')[-1].split('.')[0]) + '.avi'
writer = None
print('INFO: Read video file')
print('INFO: Output file name:', output_name)
vs = cv2.VideoCapture(video_name)
fps = vs.get(cv2.CAP_PROP_FPS)
width = int(vs.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(vs.get(cv2.CAP_PROP_FRAME_HEIGHT))
print('INFO: Video (W,H, FPS)', (width, height, fps))
max_frame = int(vs.get(cv2.CAP_PROP_FRAME_COUNT))
frame_id = 0
for i in tqdm(range(max_frame)):
  #read the next frame from file
  (grabbed, frame) = vs.read()
  if not grabbed:
    print('[INFO] Video '+ video_name +' Compeleted!!!')
    break
  try:
    frame_id += 1
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    #print(frame.shape)
  except Exception as e:
    print(e)
    break
  if i % (fps//10) == 0:
    #make prediction
    input_tensor = tf.convert_to_tensor(
        np.expand_dims(frame, 0), dtype=tf.float32)
    detections, predictions_dict, shapes = detect_fn(input_tensor)

    label_id_offset = 1
    image_np_with_detections = frame.copy()
    #nms
    nms_boxes = tf.image.non_max_suppression(detections['detection_boxes'][0], detections['detection_scores'][0],4, iou_threshold=0.3)
    nms_boxes = nms_boxes.numpy()

    bboxes = get_bboxes(detections['detection_boxes'][0].numpy()[nms_boxes],
                        detections['detection_scores'][0].numpy()[nms_boxes],
                        width, height,
                        threshold=0.4)
    cv2.line(image_np_with_detections, (width//2, 0), (width//2, height), (255, 0, 0), 5)
    cv2.line(image_np_with_detections, (0, height//4), (width, height//4), (0, 255, 0), 5)
    cv2.line(image_np_with_detections, (0, 3*height//4), (width, 3*height//4), (0, 255, 0), 5)
    text_top = 'OFF'
    text_bot = 'OFF'
    for box in bboxes:
      top_left = (box[1], box[0])
      top_right = (box[3], box[0])
      bot_left = (box[1], box[2])
      bot_right = (box[3], box[2])
      if top_left[0] < width//2 and top_right[0] > width//2 \
      and top_left[1] < height//4:
        text_top = 'ON'
      if bot_left[0] < width//2 and bot_right[0] > width//2 and bot_left[1] > 3 * width//4:
        text_bot = 'ON'
      cv2.rectangle(image_np_with_detections, (box[1], box[0]), (box[3], box[2]), (0, 0, 255), 3)
    #On or Off sprayer
    cv2.putText(image_np_with_detections, text_top, (0,300), cv2.FONT_HERSHEY_SIMPLEX,	3, (255, 0 ,0), 7)
    cv2.putText(image_np_with_detections, text_bot, (0, height-200), cv2.FONT_HERSHEY_SIMPLEX,	3, (255, 0 ,0), 7)
    #save video
    if writer is None:
      fourcc = cv2.VideoWriter_fourcc(*'MJPG')
      writer = cv2.VideoWriter(output_name, fourcc, 10, (image_np_with_detections.shape[1], image_np_with_detections.shape[0]), True)
    if writer is not None:
      image_np_with_detections = cv2.cvtColor(image_np_with_detections, cv2.COLOR_RGB2BGR)
      writer.write(image_np_with_detections)
writer.release()

INFO: Read video file
INFO: Output file name: /content/gdrive/My Drive/sprayer_robot/output_videos/video2.avi
INFO: Video (W,H, FPS) (1080, 1920, 60.0)
